# Causal ML, Uplift Modeling Part 1

## Import Libraries & Data

In [20]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklift.metrics import uplift_at_k,qini_auc_score
from sklift.datasets import fetch_hillstrom
from xgboost import XGBClassifier
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBClassifier
import optuna
from sklearn.metrics import roc_auc_score, log_loss
import matplotlib.pyplot as plt


In [21]:
# 1. Load Hillstrom dataset
def load_hillstrom():
    dataset = fetch_hillstrom()
    df = dataset.data
    df['segment'] = dataset.treatment
    df['visit'] = dataset.target
    print(df.info())
    print(df.isna().sum())
    return df

df = load_hillstrom()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64000 entries, 0 to 63999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   recency          64000 non-null  int64  
 1   history_segment  64000 non-null  object 
 2   history          64000 non-null  float64
 3   mens             64000 non-null  int64  
 4   womens           64000 non-null  int64  
 5   zip_code         64000 non-null  object 
 6   newbie           64000 non-null  int64  
 7   channel          64000 non-null  object 
 8   segment          64000 non-null  object 
 9   visit            64000 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 4.9+ MB
None
recency            0
history_segment    0
history            0
mens               0
womens             0
zip_code           0
newbie             0
channel            0
segment            0
visit              0
dtype: int64


## Explore Data

In [22]:
# The treatment
df['segment'].value_counts()

segment
Womens E-Mail    21387
Mens E-Mail      21307
No E-Mail        21306
Name: count, dtype: int64

In [23]:
# There appears to be 2 segments, let's just use the mens email to start
df = df.loc[df['segment'].isin(['Mens E-Mail','No E-Mail'])]
df['treatment'] = df['segment'].map({'Mens E-Mail':1,'No E-Mail':0})
df['treatment'].value_counts(normalize=True)

treatment
1    0.500012
0    0.499988
Name: proportion, dtype: float64

In [24]:
# This dataset has visit, conversion, and revenue as the target variables, we are going to use visit as the target variable
df['target'] = df['visit'].copy()
df['target'].value_counts()

target
0    36457
1     6156
Name: count, dtype: int64

In [25]:
df['mens'].value_counts()

mens
1    23526
0    19087
Name: count, dtype: int64

In [26]:
df['womens'].value_counts()

womens
1    23417
0    19196
Name: count, dtype: int64

In [27]:
df['history_segment'].value_counts()

history_segment
1) $0 - $100        15336
2) $100 - $200       9527
3) $200 - $350       8134
4) $350 - $500       4221
5) $500 - $750       3249
6) $750 - $1,000     1266
7) $1,000 +           880
Name: count, dtype: int64

In [28]:
df['history'].describe()

count    42613.000000
mean       241.859315
std        256.574723
min         29.990000
25%         64.500000
50%        157.000000
75%        325.210000
max       3345.930000
Name: history, dtype: float64

In [29]:
df['recency'].value_counts()

recency
1     5934
2     5074
10    5022
9     4330
3     3899
4     3406
6     3048
5     2985
7     2720
8     2337
11    2316
12    1542
Name: count, dtype: int64

In [30]:
df['newbie'].value_counts()

newbie
1    21381
0    21232
Name: count, dtype: int64

In [31]:
df['channel'].value_counts()

channel
Web             18863
Phone           18567
Multichannel     5183
Name: count, dtype: int64

In [32]:
df['zip_code'].value_counts()

zip_code
Surburban    19126
Urban        17105
Rural         6382
Name: count, dtype: int64

In [33]:
df = pd.get_dummies(df, columns=['zip_code'], drop_first=True, dtype=int)  # Encode categorical variable
df = pd.get_dummies(df, columns=['channel'], drop_first=True, dtype=int)  # Encode categorical variable
df = df.drop(columns=['history_segment','segment','visit'])

In [34]:
df.head()

,recency,history,mens,womens,newbie,treatment,target,zip_code_Surburban,zip_code_Urban,channel_Phone,channel_Web
1,6,329.08,1,1,1,0,0,0,0,0,1
3,9,675.83,1,0,1,1,0,0,0,0,1
8,9,675.07,1,1,1,1,0,0,0,1,0
13,2,101.64,0,1,0,1,1,0,1,0,1
14,4,241.42,0,1,1,0,0,0,0,0,0


In [35]:
# It appears everything is just about equal in terms of the treatment group and the control group for feature means, except the target which is ok
df.groupby('treatment').mean()

# Even if it wasn't, we could still run the model using the covariates as features, but would need to adjust for the imbalance in the treatment groups

,recency,history,mens,womens,newbie,target,zip_code_Surburban,zip_code_Urban,channel_Phone,channel_Web
treatment,,,,,,,,,,
0,5.749695,240.882653,0.553224,0.547639,0.501971,0.106167,0.451751,0.400920,0.437764,0.439923
1,5.773642,242.835931,0.550946,0.551415,0.501525,0.182757,0.445910,0.401887,0.433660,0.445394


## Split Train Test

In [36]:
# 2. Split data into train/test
def split_data(df):
    X = df.drop(columns=['treatment', 'target'])
    y = df['target']
    treatment = df['treatment']
    return train_test_split(X, y, treatment, test_size=0.3, random_state=42)

X_train, X_test, y_train, y_test, t_train, t_test = split_data(df)

## Run Two Model Approach, the traditional uplift model approach

### Build 1st Model, Control

In [50]:
# 3. Model Optimization with Optuna
def optimize_model(trial, X, y, model_type):
    if model_type == 'xgboost':
        # Set the hyperparameters to optimize and the ranges
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 5),
        }
        model = XGBClassifier(**params, eval_metric='logloss')
    elif model_type == 'random_forest':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'max_features': trial.suggest_float('max_features', 0.6, 1.0),
        }
        model = RandomForestClassifier(**params, random_state=42)
    else:
        raise ValueError("Unsupported model type")

    scores = cross_val_score(model, X, y, cv=3, scoring='roc_auc')
    return np.mean(scores)

In [51]:
# 4. Run Optuna for Both Models
def run_optuna(X, y, model_type, n_trials=50):
    # Create a study object to maximize the AUC
    study = optuna.create_study(direction='maximize')
    # optimize the study based on the input parameters
    study.optimize(lambda trial: optimize_model(trial, X, y, model_type), n_trials=n_trials)
    print(f"Best parameters for {model_type}: {study.best_params}")
    return study.best_params

In [52]:
# 5. Train and Evaluate Models
def train_and_evaluate(X_train, X_test, y_train, y_test, params, model_type):
    if model_type == 'xgboost':
        model = XGBClassifier(**params,  eval_metric='logloss')
    elif model_type == 'random_forest':
        model = RandomForestClassifier(**params, random_state=42)
    else:
        raise ValueError("Unsupported model type")

    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_pred)
    logloss = log_loss(y_test, y_pred)
    print(f"{model_type} AUC: {auc:.4f}, Log Loss: {logloss:.4f}")
    return model

In [53]:
# 6. Optimize and Train Separate Models for Two-Model Approach
def two_model_approach_with_optuna(X_train, X_test, y_train, y_test, t_train):
    
    # Use function run_optuna to optimize the treatment model for xgboost and random forest adn return the optimal hyperparameters

    # Optimize treatment model for xgboost and random forest
    X_treatment = X_train[t_train == 1]
    y_treatment = y_train[t_train == 1]
    params_treatment_xgboost = run_optuna(X_treatment, y_treatment, 'xgboost')
    params_treatment_randomforest = run_optuna(X_treatment, y_treatment, 'random_forest')

    # Optimize control model for xgboost and random forest
    X_control = X_train[t_train == 0]
    y_control = y_train[t_train == 0]
    params_control_xgboost = run_optuna(X_control, y_control, 'xgboost')
    params_control_randomforest = run_optuna(X_control, y_control, 'random_forest')

    # Train final models using function train_and_evaluate
    model_treatment_xgboost = train_and_evaluate(X_treatment, X_test, y_treatment, y_test, params_treatment_xgboost, 'xgboost')
    model_treatment_randomforest = train_and_evaluate(X_treatment, X_test, y_treatment, y_test, params_treatment_randomforest, 'random_forest')
    model_control_xgboost = train_and_evaluate(X_control, X_test, y_control, y_test, params_control_xgboost, 'xgboost')
    model_control_randomforest = train_and_evaluate(X_control, X_test, y_control, y_test, params_control_randomforest, 'random_forest')

    return model_treatment_xgboost, model_control_randomforest, model_treatment_randomforest, model_control_xgboost

In [54]:
# Execute the Two-Model Approach with Optuna usng function "two_model_approach_with_optuna"
model_treatment_xgboost, model_control_randomforest, model_treatment_randomforest, model_control_xgboost = two_model_approach_with_optuna(X_train, X_test, y_train, y_test, t_train)

[I 2024-12-19 20:43:10,616] A new study created in memory with name: no-name-3ec8e898-dd10-469f-aba3-5bb99b633e6c
C:\Users\wallj\AppData\Local\Temp\ipykernel_7600\3930955610.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
[I 2024-12-19 20:43:10,935] Trial 0 finished with value: 0.61238470912025 and parameters: {'n_estimators': 207, 'max_depth': 4, 'learning_rate': 0.02047579304694242, 'subsample': 0.8860735617267557, 'colsample_bytree': 0.9328120967876155, 'gamma': 3.395245184765317}. Best is trial 0 with value: 0.61238470912025.
C:\Users\wallj\AppData\Local\Temp\ipykernel_7600\3930955610.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0.

Best parameters for xgboost: {'n_estimators': 208, 'max_depth': 10, 'learning_rate': 0.19969570808437617, 'subsample': 0.7480345228273033, 'colsample_bytree': 0.6007327056030227, 'gamma': 4.974775486769139}


[I 2024-12-19 20:43:29,036] Trial 0 finished with value: 0.582290759626645 and parameters: {'n_estimators': 105, 'max_depth': 14, 'min_samples_split': 9, 'min_samples_leaf': 10, 'max_features': 0.7703224730290333}. Best is trial 0 with value: 0.582290759626645.
[I 2024-12-19 20:43:30,975] Trial 1 finished with value: 0.5867509469977469 and parameters: {'n_estimators': 82, 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': 0.7035596898497694}. Best is trial 1 with value: 0.5867509469977469.
[I 2024-12-19 20:43:39,198] Trial 2 finished with value: 0.5634457775921785 and parameters: {'n_estimators': 222, 'max_depth': 20, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 0.9383178399292939}. Best is trial 1 with value: 0.5867509469977469.
[I 2024-12-19 20:43:40,823] Trial 3 finished with value: 0.604271230989664 and parameters: {'n_estimators': 86, 'max_depth': 6, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 0.7884469129018556}. 

Best parameters for random_forest: {'n_estimators': 292, 'max_depth': 5, 'min_samples_split': 8, 'min_samples_leaf': 4, 'max_features': 0.6492393174505139}


[I 2024-12-19 20:46:49,311] Trial 0 finished with value: 0.6200515358896064 and parameters: {'n_estimators': 230, 'max_depth': 9, 'learning_rate': 0.0212402726568262, 'subsample': 0.7259614626563833, 'colsample_bytree': 0.8758441425988741, 'gamma': 0.7407317326689533}. Best is trial 0 with value: 0.6200515358896064.
C:\Users\wallj\AppData\Local\Temp\ipykernel_7600\3930955610.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
[I 2024-12-19 20:46:49,734] Trial 1 finished with value: 0.6240923798408509 and parameters: {'n_estimators': 276, 'max_depth': 7, 'learning_rate': 0.06644251062462836, 'subsample': 0.8475617349895112, 'colsample_bytree': 0.8562976150738247, 'gamma': 1.0549534469284754}. Best is trial 1 with value: 0.6240923798408509.
C:\Users\wallj\AppD

Best parameters for xgboost: {'n_estimators': 209, 'max_depth': 3, 'learning_rate': 0.16329003519969132, 'subsample': 0.6975782227141417, 'colsample_bytree': 0.6326926782114536, 'gamma': 3.779263400491446}


[I 2024-12-19 20:47:09,030] Trial 0 finished with value: 0.6317075993926534 and parameters: {'n_estimators': 278, 'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 0.9060167842798403}. Best is trial 0 with value: 0.6317075993926534.
[I 2024-12-19 20:47:10,993] Trial 1 finished with value: 0.5951366672837264 and parameters: {'n_estimators': 63, 'max_depth': 14, 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_features': 0.8063960291176899}. Best is trial 0 with value: 0.6317075993926534.
[I 2024-12-19 20:47:17,504] Trial 2 finished with value: 0.6028790272124743 and parameters: {'n_estimators': 230, 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 7, 'max_features': 0.880295376453907}. Best is trial 0 with value: 0.6317075993926534.
[I 2024-12-19 20:47:19,146] Trial 3 finished with value: 0.6009722173544033 and parameters: {'n_estimators': 60, 'max_depth': 20, 'min_samples_split': 3, 'min_samples_leaf': 8, 'max_features': 0.8794890893163934}. 

Best parameters for random_forest: {'n_estimators': 178, 'max_depth': 5, 'min_samples_split': 3, 'min_samples_leaf': 4, 'max_features': 0.6205596835720937}
xgboost AUC: 0.6228, Log Loss: 0.4094
random_forest AUC: 0.6198, Log Loss: 0.4099
xgboost AUC: 0.6198, Log Loss: 0.4145


c:\Users\wallj\anaconda3\envs\pymc_env\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:49:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


random_forest AUC: 0.6166, Log Loss: 0.4157


In [18]:
# 3. Two-Model Approach
def two_model_approach(X_train, y_train, t_train, X_test):
    model_treatment = RandomForestClassifier(random_state=42)
    model_control = RandomForestClassifier(random_state=42)

    # Train separate models for treatment and control
    model_treatment.fit(X_train[t_train == 1], y_train[t_train == 1])
    model_control.fit(X_train[t_train == 0], y_train[t_train == 0])

    # Predict probabilities
    uplift_treatment = model_treatment.predict_proba(X_test)[:, 1]
    uplift_control = model_control.predict_proba(X_test)[:, 1]

    uplift = uplift_treatment - uplift_control
    return uplift

uplift_two_model = two_model_approach(X_train, y_train, t_train, X_test)

In [19]:
# 4. S-Learner
def single_model(X_train, y_train, t_train, X_test):
    X_train_s = X_train.copy()
    X_train_s['treatment'] = t_train

    X_test_s = X_test.copy()
    X_test_s['treatment'] = 1  # Predict as if treated
    X_test_c = X_test.copy()
    X_test_c['treatment'] = 0  # Predict as if control

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train_s, y_train)

    uplift_treatment = model.predict_proba(X_test_s)[:, 1]
    uplift_control = model.predict_proba(X_test_c)[:, 1]

    uplift = uplift_treatment - uplift_control
    return uplift

uplift_s_learner = single_model(X_train, y_train, t_train, X_test)

In [ ]:
# 5. Enhanced Qini Evaluation
def evaluate_qini(uplift, y_test, t_test, model_name):
    qini = qini_auc_score(y_test, uplift, t_test)

    # Use a loop to calculate uplift at different percentages
    k_values = [i / 100 for i in range(1, 100)]  # Generate k as float values from 0.01 to 1
    uplift_cumulative = []

    for k in k_values:
        uplift_k = uplift_at_k(y_test, uplift, t_test, strategy='overall', k=k)
        uplift_cumulative.append(uplift_k)

    print(f"{model_name} Qini Score: {qini:.4f}")

    # Plot the Qini Curve
    plt.plot(k_values, uplift_cumulative, label=f'{model_name} Model')
    plt.plot([0, 1], [0, max(uplift_cumulative)], '--', label='Random')
    plt.xlabel('Proportion of Population Targeted')
    plt.ylabel('Cumulative Uplift')
    plt.title(f'Qini Curve - {model_name}')
    plt.legend()
    plt.show()

# Compare Two-Model and S-Learner
print("### Two-Model Approach Evaluation ###")
evaluate_qini(uplift_two_model, y_test, t_test, "Two-Model")

print("### S-Learner Evaluation ###")
evaluate_qini(uplift_s_learner, y_test, t_test, "S-Learner")


In [ ]:
# 6. Tabular Comparison of Qini Scores
qini_two_model = qini_auc_score(y_test, uplift_two_model, t_test)
qini_s_learner = qini_auc_score(y_test, uplift_s_learner, t_test)

comparison_df = pd.DataFrame({
    'Model': ['Two-Model', 'S-Learner'],
    'Qini Score': [qini_two_model, qini_s_learner]
})
print("\n### Qini Score Comparison ###")
print(comparison_df)